In [1]:
import os

dir_path = r"C:\Users\Aditya\Github\zavmo\zavmo-api\zavmo"
os.chdir(dir_path)

In [22]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
#path = r"C:\Users\Aditya\GitHub\zavmo\zavmo-api\docs\nos-ofqual\parsed_nos_data.xlsx"
path = r"C:\Users\Aditya\GitHub\zavmo\zavmo-api\docs\nos-ofqual\parsed_nos2.xlsx"

In [4]:
data = pd.read_excel(path)

In [5]:
def combine_text(df):
    nos_id   = df['nos_id'].unique()[0]
    title    = df['title'].unique()[0]
    industry = df['industry'].unique()[0]
    text     = f"Document Name: {nos_id}\nIndustry: {industry}\nTitle: {title}\n"
    text    += '\n'.join([f"**{row['type']}**: {row['text']}" for idx, row in df.iterrows()])
    return text

In [12]:
from helpers.chat import get_openai_embedding, get_batch_openai_embedding
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
query = "Energy Compliance Consultant, Compliance Department, Energy and Gas Industry"

In [28]:
query = """
Energy Compliance Consultant, Compliance Department, Energy and Gas Industry.

Functional/Technical Skills:
▪ Effective communicator in both written and verbal communictions, including
ability to propose practical actions and tailor communications to a variety of
levels of stakeholder (including senior management and the Board).
▪ Confident in absorbing and manipulating information/data, from multiple
sources (and multiple formats, e.g Microsoft Excel, PowerBI, operational
systems), to drive Assurance insight, review conclusions and proportionate
actions.
▪ Able to understand, identify and clearly articulate risks and controls arising in
relation to a wide range of issues, with a particular focus on conduct risk and
customer outcomes, and ability to design testing activities in line with the
department methodology.
▪ Robust and independent mindset with the confidence to raise issues
appropriately.
▪ Advanced level stakeholder skills and comfortable dealing with all grades of
colleague.
▪ Able to work effectively individually and as part of a team to deliver to tight
deadlines.

Education and Certifications:
 ▪ Demonstrable experience of compliance activity within a regulatory sector
and/or educated to degree standard (Law or Economics degree preferable)
with aspiration to work in a high performing compliance function.

Competencies:
▪ Demonstrable experience of working in a regulated environment and
understanding of the energy supply regulatory framework.
▪ Credibility to build relationships with a variety of internal stakeholders and
influence accordingly.
▪ Experience in multi-tasking, being able to navigate complex tasks & prioritise
effectively.
▪ Developing close ‘trusted adviser’ relationships with key business leaders.
▪ Wins hearts and minds across the business and inspires others to act for the
betterment of our business and its customers.
▪ Proven oral and written communication skills with the ability to explain
complex compliance requirements and prove pragmatic advice to stakeholders.
▪ Engage with stakeholders in an open, honest, and productive manner to
promote collaborative working.
▪ Effectively manage stakeholders and senior management during meetings by
adapting the style of communication to suit the situation and the individual."""

In [29]:
qv = get_openai_embedding(query)

In [30]:
nos_doc = """NOS ID: **FSPCOMP1**
- **Industry:** Compliance  
- **Title:** Identify the requirements of legislation and regulations for the business conducted by your organisation  
- **Overview:** This NOS outlines identifying regulatory bodies, assessing the impact of legislation on organizational policies, determining compliance requirements, and reporting findings to appropriate stakeholders.  
- **Performance Criteria:**  
  1. Identify applicable regulatory bodies and relevant legislation.  
  2. Assess the impact of regulations on organizational policies.  
  3. Determine compliance requirements and necessary resources.  
  4. Report compliance assessments to relevant stakeholders.  

- **Knowledge & Understanding Required:**  
  1. Legal and regulatory frameworks relevant to the organization.  
  2. Compliance and resourcing implications.  
  3. Consequences of non-compliance.  
  4. Impact of regulations on different business functions.  

- **Keywords:** Legislation, Regulations, Compliance, Non-compliance, Governance, Strategy.  
- **Relevant Roles:** Finance Managers, Compliance Officers, Regulatory Specialists.
"""

In [31]:
nos_vec = get_openai_embedding(nos_doc)

In [32]:
qv_reshaped = np.array(qv).reshape(1, -1)
nos_vec_reshaped = np.array(nos_vec).reshape(1, -1)

# Compute cosine similarity
similarity = cosine_similarity(qv_reshaped, nos_vec_reshaped)

In [33]:
similarity

array([[0.53332771]])